In [ ]:
!pip install langchain openai chromadb tiktoken pypdf langchain_openai
!pip install -U langchain-community

# Env preparation

In [ ]:
## Overview
import os
import openai
import sys
sys.path.append('../..')

### Prompt
os.environ["OPENAI_API_KEY"] = "<<openai-key>>" # replace dots with your api key


# Document loader

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

def load_pdf():
  loader = PyPDFLoader("docs/spaceport.pdf")
  return loader.load()

# Document splitting

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_doc(doc):
  splitter = RecursiveCharacterTextSplitter(
      chunk_size=300,
    chunk_overlap=100
  )

  return splitter.split_documents(doc)


# Embedding
This sections creates an embedding and stores it to the chroma db

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

openai_embedding = OpenAIEmbeddings()

vectordb = Chroma(
    persist_directory='chroma/',
    embedding_function=openai_embedding
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
def persist_chroma(docs):
  vectordb.add_documents(docs)

# QA Retriveral Chain

In [ ]:
openai_model = 'gpt-3.5-turbo'

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model=openai_model, temperature=0)

## Chat memory implementation
This implements the `BaseChatMessageHistory` class to store the memory.


In [106]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.memory import ConversationBufferWindowMemory

class BufferChatWindowMemory(BaseChatMessageHistory):
  window_memory = ConversationBufferWindowMemory(k=5, return_messages=True)

  @property
  def messages(self):
    return self.window_memory.load_memory_variables({})["history"]

  def add_messages(self, messages):
      self.window_memory.save_context({"input": messages[0].content}, {"output": messages[1].content})

  def clear(self):
    self.window_memory.clear()

### Messages template

In [107]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts.chat import MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.
Respond in the same language as the user.

<context>
{context}
</context>
"""

prompt = ChatPromptTemplate.from_messages([
      ("system", SYSTEM_TEMPLATE),
      MessagesPlaceholder(variable_name="chat_history"),
      ("human", "{input}"),
])

## Chatbot chain creator

In [108]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def create_chain(
    retriever,
    prompt,
    llm_model
):
  return (
      RunnableParallel(
          context=(lambda x: x['input']) | retriever | format_docs,
          input=lambda x: x['input'],
          chat_history=lambda x: x['chat_history']
      )
      | prompt
      | llm_model
  )

# Running

### Loading pdf onto vector db

In [109]:
# Load and persist into chroma db
pdf_load = load_pdf()
docs = split_doc(pdf_load)

persist_chroma(docs)

### Chatbot chain

In [110]:
chroma_retriever = vectordb.as_retriever()

cb_chain = create_chain(
    chroma_retriever,
    prompt,
    model
)

## Runnable with memory
This enables memory saving in context window

In [111]:
from langchain_core.runnables.history import RunnableWithMessageHistory

memory = BufferChatWindowMemory()

chain_with_message_history = RunnableWithMessageHistory(
    cb_chain,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [114]:
chain_with_message_history.invoke({"input": "What's document about ?"}, {"configurable": {"session_id": "unset"}})

AIMessage(content='El documento es un White Paper sobre el "Ecuador Spaceport" y destaca la importancia de la tecnología espacial como un factor fundamental para la competitividad de la economía ecuatoriana. ¿Hay algo más que te gustaría saber sobre el documento?', response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 407, 'total_tokens': 462}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1fa4f455-df09-4a53-94b1-98eca8e79a48-0', usage_metadata={'input_tokens': 407, 'output_tokens': 55, 'total_tokens': 462})

# Test for window memory Chatbot

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages([
      (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
      ),
      MessagesPlaceholder(variable_name="chat_history"),
      ("human", "{input}"),
])

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

chain = prompt | llm

memory = BufferChatWindowMemory()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

AIMessage(content='Hello Juan! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 35, 'total_tokens': 45}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_b81a85d4e3', 'finish_reason': 'stop', 'logprobs': None}, id='run-c6d53833-6d28-40f7-8fbb-a27f221b61ea-0', usage_metadata={'input_tokens': 35, 'output_tokens': 10, 'total_tokens': 45})

In [ ]:
chain_with_message_history.invoke({"input": "Hello assistant, my name is juan"}, {"configurable": {"session_id": "ss"}})